In [1]:
import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from srgan2 import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# !TF_ENABLE_GPU_GARBAGE_COLLECTION=false

In [3]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [4]:
input_dir = 'data/train/low/'
tgt_dir = 'data/train/high/'

input_dir_val = 'data/val/low/'
tgt_dir_val = 'data/val/high/'


test_file = 'UBMk30rjy0o_17675_1.jpg'
test_low_path =  input_dir_val + test_file
test_high_path = tgt_dir_val + test_file
test_out_path = 'data/out/' + test_file
test_test_path = 'data/test/' + test_file

vgg_path = 'models/vgg16_notop.hdf5'

input_shape = (270, 480, 3)
tgt_shape = (540, 960, 3)

batch_size = 1

In [5]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)
gen_valid = ImageDataGenerator(input_dir_val, tgt_dir_val, batch_size)

/home/keras/srgan2.py:114: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="generator", inputs=Tensor("in..., outputs=Tensor("ou...)`
  self.shared_generator = Network(input=layers[0], output=layers[-1], name='generator')
/home/keras/srgan2.py:141: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="discriminator", inputs=Tensor("in..., outputs=Tensor("de...)`
  self.shared_discriminator = Network(input=layers[0], output=layers[-1], name='discriminator')
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
for e in range(10001):
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)
    if e % 100 == 0:
        print('epoch', e)
        srgan.valid(X_low, X_high)
        
        test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
        test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

        test_img_sr = srgan.generator.predict(test_img_low)
        test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
        test_out_path = 'data/out/'  + test_file.split('.')[0] + f'_test_{e}.jpg'
        cv2.imwrite(test_out_path, test_img_sr_int)

epoch 0
d_loss_real:0.45005810260772705, d_loss_fake:0.9880667924880981, vgg_loss:7.561858654022217
epoch 100
d_loss_real:0.7159875631332397, d_loss_fake:0.6700860857963562, vgg_loss:8.77290153503418


In [ ]:
srgan.generator.save('models/20200512_generator.h5', include_optimizer=False)

In [ ]:
from keras.models import Model, load_model
from keras.engine.network import Network
from keras.layers import Input

# pre_model = load_model('20200510_generator.h5')
pre_model = srgan.generator
pre_net = Network(inputs=pre_model.input, outputs=pre_model.get_layer(pre_model.layers[-1].name).output, name='learned')

input_layer = Input(shape = (540, 960, 3))
sr_out = pre_net(input_layer)
sr_generator = Model(input_layer, sr_out)


In [ ]:
img_list = []
for i in range(4):
    test_file = f'UBMk30rjy0o_17675_{i}.jpg'
    path = input_dir_val + test_file
    test_img_low = ((cv2.imread(path) - 127.5) / 127.5)
    img_list.append(test_img_low)

h0 = np.vstack([img_list[0], img_list[1]])
h1 = np.vstack([img_list[2], img_list[3]])
img = np.hstack([h0, h1])
img_gen = sr_generator.predict(img.reshape((1, 540, 960, 3)))

img_int = ((img_gen * 127.5) + 127.5).astype('u1').reshape((1080, 1920, 3))
cv2.imwrite('data/test/20200511_out_premerged.jpg', img_int)